In [2]:
with open('places.json', 'r', encoding='utf-8') as f:
    import json
    places = json.load(f)
len(places)

description_to_rate_mapping = {
    'взглянуть': 3, 'увидеть интересно': 2, 'увидеть обязательно': 1, 'увидеть по дороге': 4
}

prop_key_name_correction_mapping = {
    'Даты строи-тельства': 'Даты строительства',
    "Дата строи-тельства": 'Даты строительства / обновления',
    'Гос. цен-ность (0..3)': 'Гос. ценность (0..3)',
    "Рейтинг(1..4)": "Рейтинг (1..4)",
}

date_property_names = [
    'Дата',
    'Дата установки',
    'Дата строительства',
    'Даты строительства',
    'Даты / обновления',
    'Даты строительства',
    'Даты строительства / обновления',
    'Дата основания'
]

def get_sight_date(props):
    for date_property_name in date_property_names:
        if date_property_name in props:
            return props[date_property_name]
    return None

def get_sight_prop_value(props, prop_name):
    if prop_name not in props:
        return None
    return props[prop_name]


def convert_props_to_dict(props):
    props = {}
    for prop_key, value in sight['props']:
        if prop_key in prop_key_name_correction_mapping:
            prop_key = prop_key_name_correction_mapping[prop_key]
        props[prop_key] = value
    return props


def get_coords(props):
    coord = get_sight_prop_value(props, 'Координаты')
    if coord:
            return (coord, True)
    else:
        potential_coord = get_sight_prop_value(props, 'Предполагаемые  координаты')
        if potential_coord:
            return (potential_coord, True)
    return None

def get_sight_rate(props):
    rate = get_sight_prop_value(props, 'Рейтинг (1..4)')
    if rate:
        rate = int(description_to_rate_mapping[rate])
    return rate


def get_sight_categories(props):
    tags = []
    tag_list = get_sight_prop_value(props, 'Списки')
    if tag_list:
        tags = split_tags(tag_list)
    return tags


def get_sight_image_url(sight):
    image_url = sight['image']
    if image_url:
        start_index = image_url.rindex('https')
        image_url = image_url[start_index:]
        sight['image'] = image_url
    return image_url


import re

tags_to_remove = ['Топ 100', 'Топ 20']

def split_tags(tag_list):
    for tag in tags_to_remove:
        tag_list = tag_list.replace(tag, '')
    # Regex pattern to match uppercase Cyrillic letters
    pattern = r'(?<=[а-яё\.\)\d])([А-ЯЁ])'

    # Insert a separator character before each uppercase Cyrillic letter except the first one
    modified_string = re.sub(pattern, r'|\1', tag_list)

    pattern = r'(?<=[а-яё\)])([1-9])'

    modified_string = re.sub(pattern, r'|\1', modified_string)

    # Split the modified string using the separator character to get the list of items
    result = modified_string.split('|')

    return result


sight_id = 0
sights_db = []
for i, p in enumerate(places):
    p['coords'] = p['info']['coords']
    p['location'] = p['info']['location']
    if 'name' in p['info']:
        p['name'] = p['info']['name']
    p['sights'] = p['info']['sights']
    p['rating'] = int(p['rating'])
    p['id'] = i
    del p['info']
    for sight in p['sights']:
        sight['id'] = sight_id
        sight_id += 1
        
        sight['image'] = get_sight_image_url(sight)

        props = convert_props_to_dict(sight['props'])
        sight['coord'] = get_coords(props)
        sight['hint'] = get_sight_prop_value(props, 'Ориентир')
        sight['date'] = get_sight_date(props)
        sight['rate'] = get_sight_rate(props)
        sight['state_rate'] = get_sight_prop_value(props, 'Гос. ценность (0..3)')
        sight['status'] = get_sight_prop_value(props, 'Статус')
        sight['address'] = get_sight_prop_value(props, 'Адрес')
        sight['style'] = get_sight_prop_value(props, 'Стиль')
        sight['tags'] = get_sight_categories(props)
        sight['props'] = props

In [4]:
sights = []
for p in places:
    place_pointer = {
            'id': p['id'],
            'name': p['name'],
            'rating': p['rating'],
            'location': p['location'],
            'coords': p['coords']
        }
    for sight in p['sights']:
        sight_record = dict(sight)
        sight_record['place'] = place_pointer
        sights.append(sight_record)

In [5]:
with open('places_db.json', 'w', encoding='utf-8') as f:
    json.dump({"places": places, 'sights': sights}, f, indent=4, ensure_ascii=False)

In [42]:
categories = set()
for place in places:
    for sight in place['sights']:
        for category in sight['tags']:
            categories.add(category)
categories

{'',
 '1-я мировая война',
 '1-я мировая война (?)',
 'Аптеки',
 'Арочные мосты',
 'Банки и казначейства',
 'Башни',
 'Башни оборонительного типа',
 'Борисовы камни',
 'Брамы',
 'Брамы (?)',
 'Брамы (избранное)',
 'Бровары и винокурни',
 'Бровары и винокурни (?)',
 'Ветряные мельницы',
 'Ветряные электрогенераторы',
 'Водяные мельницы',
 'Водяные мельницы (?)',
 'Вокзалы и ж/д станции',
 'Восстание 1830-31 гг.',
 'Восстание 1863-64 гг.',
 'Восстание 1863-64 гг. (?)',
 'Восстание Костюшко 1794 г.',
 'Гаштольды',
 'Гаштольды (?)',
 'Гмины, староства, воеводства',
 'Городница',
 'Гостиницы и заезжие дворы',
 'Греко-католические церкви действующие',
 'Дворцы',
 'Дворцы (?)',
 'Деревянное зодчество (избранное)',
 'Деревянные костелы (избранное)',
 'Деревянные церкви (избранное)',
 'Дом-крепость',
 'Дома-музеи',
 'Дуга Струве',
 'Еврейское наследие',
 'Еврейское наследие (?)',
 'Заброшенные и захороненные деревни',
 'Замки',
 'Иешивы',
 'Иешивы (?)',
 'Интересная география',
 'Интересные над

In [10]:
for place in places:
    for sight in place['sights']:
        date_props = [p for p in sight['props'] if 'дат' in p.lower()]
        if len(date_props) > 1:
            print(sight['name'])
            print(sight['props'])
            print()

In [84]:
sight_to_rate = []
for place in places:
    rate = place['rating']
    for sight in place['sights']:
        if not sight['rate']:
            sight_rate = rate * 4
        else:
            sight_rate = sight['rate'] * rate
        sight_to_rate.append((sight, sight_rate, place))

In [85]:
sorted_sights = sorted(sight_to_rate, key=lambda triple: triple[1])
for sight, rate, place in sorted_sights[:10]:
    print(place['name'], rate)
    print(json.dumps(sight, indent=4, ensure_ascii=False))
    print()

Ружаны 1
{
    "name": "дворцовый комплекс Сапег",
    "image": "https://orda.of.by/.ga/r/ruzhany/palace/nf/ruzhany_12.jpg",
    "coord": [
        "52.860723, 24.895983 (*)",
        true
    ],
    "hint": null,
    "building_date": "1598—1605 гг., 1748—86 гг.",
    "foundation_date": null,
    "rate": 1,
    "state_rate": null,
    "status": null,
    "address": null,
    "style": "переход от позднего барокко к классицизму",
    "tags": [
        "Топ 20",
        "Топ 100",
        "Объекты 1600-1699 гг.",
        "Дворцы",
        "Усадьбы и фольварки",
        "Сапеги"
    ]
}

Жиличи 1
{
    "name": "усадьба Булгаков \"Добосна\"",
    "image": null,
    "coord": null,
    "hint": null,
    "building_date": "XVIII—XIX вв.",
    "foundation_date": null,
    "rate": 1,
    "state_rate": null,
    "status": null,
    "address": null,
    "style": null,
    "tags": [
        "Усадьбы и фольварки"
    ]
}

Жиличи 1
{
    "name": "усадьба: дворец",
    "image": "https://globustut.by/zh

In [18]:
sight_props = set()
for place in places:
    for sight in place['sights']:
        for prop_key, value in sight['props'].items():
            sight_props.add(prop_key)
sight_props

{'Адрес',
 'Гос. ценность (0..3)',
 'Дата',
 'Дата основания',
 'Дата строительства',
 'Дата установки',
 'Даты / обновления',
 'Даты строительства',
 'Даты строительства / обновления',
 'Координаты',
 'На старых картах',
 'Ориентир',
 'Предполагаемые  координаты',
 'Рейтинг (1..4)',
 'Списки',
 'Статус',
 'Стиль'}

In [20]:
def display_possible_values(prop_name):
    values = set()
    for place in places:
        for sight in place['sights']:
            if prop_name not in sight['props']:
                continue
            values.add(sight['props'][prop_name])
    return values

In [25]:
for sight_prop in sight_props:
    values = list(display_possible_values(sight_prop))
    print(sight_prop, len(values))
    print(values[:5])
    print()

Координаты 3125
['54.586959, 29.740172 (*)', '54.366383, 26.661196 (*)', '53.246187, 24.241047 (*)', '54.427397, 25.934637 (*)', '54.084465, 30.768924 (*)']

Дата 217
['1975 г.', 'XVI в.', '1920 г.', '1974 г.', 'XVIII—XX вв.']

Даты строительства 10
['1597—98 гг.? / XVII в. / XIX в.', '1643—46 гг. / 2-я половина XVIII в.', '1637 г.—2-я половина XVII в. / 1746—50 гг.', '1830-е гг. / 1864—76 гг.', '1750 г. / 1830—35 гг.']

Адрес 401
['ул. Энгельса', 'ул. Лопатина, 38', 'ул. Ленинская, 59', 'ул. Ленина', 'ул. Маркса, 29']

Дата основания 57
['основ. 1-я половина XVII в.?', 'основ. 1992 г.', 'основ. 1128 г.', 'основ. 1883 г.', 'основ. 1628 г.']

Даты / обновления 8
['середина XVIII в. / 1-я половина XIX в.', '1793 г. / 1830 г.', 'начало XIX в. / конец XIX в.', '1912 г. / 1962 г.', '2-я половина XVIII в. / конец XIX—начало XX вв.']

Ориентир 1101
['на деревенском кладбище, в центре кладбища', 'позади часовни-усыпальницы', 'на северо-восточной окраине города', 'между Мяделем и дер. Бояры, в 